# Install the Library

In [ ]:
!pip install interpret-pytorch

# Generate Visualisations from a Pretrained VGG

In [ ]:
from interpret import OptVis, ImageParam, denorm
import torchvision, torch

network = torchvision.models.vgg11(pretrained=True)
network.to('cuda' if torch.cuda.is_available() else 'cpu');

## Class Visualisations

First we generate visualisations of the classes. To change the class selected, simply change the value of `neuron`.

We parameterise the input noise in the colour decorrelated, Fourier domain. This helps create better visualisations. For more, see: https://distill.pub/2017/feature-visualization/

In [ ]:
# Select a layer from the network. Use get_layer_names() to see a list of layer names and sizes.
layer = 'classifier/6'
neuron = 5

# Create an OptVis object from a PyTorch model
optvis = OptVis.from_layer(network, layer=layer, neuron=neuron)

# Parameterise input noise in colour decorrelated Fourier domain
img_param = ImageParam(128, fft=True, decorrelate=True)

# Create visualisation
optvis.vis(img_param, thresh=(250, 500), transform=True, lr=0.05, wd=5.)

## Channel Visualisations

Now let's generate some visualisations of the channels of the convolutional layers of the network. We can see the names and number of channels of each layer by using the `get_layer_names()` method. We can then generate channel visualisations in the same way.

In [ ]:
from interpret import get_layer_names

In [ ]:
get_layer_names(network)

In [ ]:
# Select a layer from the network. Use get_layer_names() to see a list of layer names and sizes.
layer = 'features/16'

# Choose a channel that is within the size of the layer
channel = 32

# Create an OptVis object from a PyTorch model
optvis = OptVis.from_layer(network, layer=layer, channel=channel)

# Parameterise input noise in colour decorrelated Fourier domain
img_param = ImageParam(224, fft=True, decorrelate=True)

# Create visualisation
optvis.vis(img_param, thresh=(250, 500), transform=True, lr=0.05, wd=0.03)

# Generate Attribution maps Using Grad-CAM

Grad-CAM [1] is a technique that finds relevant features for a particular class. The method generates a heatmap over the input where the network identifies features of that particular class.

[1] - https://arxiv.org/abs/1610.02391

In [ ]:
from interpret import gradcam, norm
from PIL import Image

In [ ]:
# Download an image to apply attribution to
!curl https://www.yourpurebredpuppy.com/dogbreeds/photos2-G/german-shepherd-05.jpg -o dog.jpg

In [ ]:
img = Image.open("dog.jpg")
img

In [ ]:
data = norm(img).to('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class_number = 235 # German Shepherd
layer = 'features/20'

gradcam(network, data, im_class=class_number, layer=layer);

In [ ]:
# You can also show the heatmap without the original image to see the heatmap clearer
gradcam(network, data, im_class=class_number, layer=layer, show_im=False);